In [1]:
!pip install ops

In [1]:
!pip install --upgrade keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [2]:
import warnings
import logging
import os
warnings.filterwarnings('ignore')
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import keras
from tensorflow.keras import ops, layers


SEED = 999
AUTO = tf.data.AUTOTUNE
tf.random.set_seed(SEED)

In [3]:
num_classes = 2
input_shape = (224, 224, 3)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Hiperparamtros para una ViTB-16:


*   learning_rate = 0.001  # Puedes ajustar según sea necesario
*   weight_decay = 0.0001


*   batch_size = 16

*   patch_size = 16

*  num_patches = (image_size // patch_size) ** 2
*   projection_dim = 768


*   num_heads = 12


*   transformer_units = [    3072,    768,]


*   transformer_layers = 12
*   mlp_head_units = [    3072,    768,]


In [5]:
learning_rate = 0.0001  # Reducción de la tasa de aprendizaje
weight_decay = 0.0001
batch_size = 16
num_epochs = 10  # Incrementar el número de épocas si se reduce la tasa de aprendizaje
image_size = 224
patch_size = 16  # Reducir el tamaño del parche para capturar más detalles
num_patches = (image_size // patch_size) ** 2
projection_dim = 128
num_heads = 6
transformer_units = [
    projection_dim * 2,
    projection_dim,
]
transformer_layers = 8  # Reducir el número de capas para simplificar el modelo
mlp_head_units = [
    1024,
    512,
]

In [6]:
# Creating main dataframe contains image paths and their classes.

def create_images_list(path):
    full_path = []
    images = sorted(os.listdir(path))
    for i in images:
        full_path.append(os.path.join(path, i))
    return full_path


train_data_0 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/train_converted_images/0')
train_data_1 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/train_converted_images/1')

val_data_0 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/val_converted_images/0')
val_data_1 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/val_converted_images/1')

test_data_0 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/test_converted_images/0')
test_data_1 = create_images_list('/content/drive/My Drive/Tesis/Proyectos2/BD_GlaucomaTorch/test_converted_images/1')


# look-up table
classes = {0: 'non-glaucoma', 1: 'glaucoma'}


train_df = pd.concat([pd.DataFrame({ 'image' : train_data_0, 'label': 0 }),
                      pd.DataFrame({ 'image' : train_data_1, 'label': 1 }) ], ignore_index = True)


val_df = pd.concat([pd.DataFrame({ 'image' : val_data_0, 'label': 0 }),
                    pd.DataFrame({ 'image' : val_data_1, 'label': 1 }) ], ignore_index = True)


test_df = pd.concat([pd.DataFrame({ 'image' : test_data_0, 'label': 0 }),
                     pd.DataFrame({ 'image' : test_data_1, 'label': 1 }) ], ignore_index = True)


train_df = shuffle(train_df, random_state = SEED).reset_index(drop = True)
val_df = shuffle(val_df, random_state = SEED).reset_index(drop = True)
test_df = shuffle(test_df, random_state = SEED).reset_index(drop = True)

# Seleccionar solo una tercera parte de los datos
# train_df = train_df.sample(frac=1/3, random_state=SEED).reset_index(drop=True)
# val_df = val_df.sample(frac=1/3, random_state=SEED).reset_index(drop=True)
# test_df = test_df.sample(frac=1/3, random_state=SEED).reset_index(drop=True)


print("train images -> ", train_df.shape[0])
print("val images -> ", val_df.shape[0])
print("test images -> ", test_df.shape[0])



train images ->  8631
val images ->  5767
test images ->  2250


In [7]:
def load_and_preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label


train_dataset = tf.data.Dataset.from_tensor_slices((train_df['image'], train_df['label']))
train_dataset = train_dataset.map(load_and_preprocess_image)
train_dataset = train_dataset.batch(batch_size)

In [8]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_df['image'], test_df['label']))
test_dataset = test_dataset.map(load_and_preprocess_image)
test_dataset = test_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((val_df['image'], val_df['label']))
val_dataset = val_dataset.map(load_and_preprocess_image)
val_dataset = val_dataset.batch(batch_size)

In [10]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        # layers.Resizing(image_size, image_size),
        # layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
        # layers.RandomTranslation(height_factor=0.1, width_factor=0.1),  # Agregar más aumento de datos
    ],
    name="data_augmentation",
)

In [11]:
def mlp(x, hidden_units, dropout_rate, l2_reg=0.002):
    for units in hidden_units:
        x = layers.Dense(units, activation=keras.activations.gelu, kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [22]:
class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(Patches, self).__init__(**kwargs)
        # super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

In [24]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super(PatchEncoder, self).__init__(**kwargs)
        #super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [14]:
def create_vit_classifier():
    inputs = keras.Input(shape=input_shape)
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim, dropout=0.1)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)

    # Nueva capa densa añadida
    features = layers.Dense(256, activation='gelu')(features)
    features = layers.Dropout(0.5)(features)
    logits = layers.Dense(num_classes)(features)
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [16]:
modelo = create_vit_classifier()
modelo.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patches (Patches)         │ (None, None, 768)      │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patch_encoder             │ (None, 196, 128)       │        123,520 │ patches[0][0]          │
│ (PatchEncoder)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 196, 128)       │            256 │ patch_encoder[0][0]    │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 196, 128)       │        395,648 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 196, 128)       │              0 │ multi_head_attention[… │
│                           │                        │                │ patch_encoder[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 196, 128)       │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 196, 256)       │         33,024 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 196, 256)       │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 196, 128)       │         32,896 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 196, 128)       │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 196, 128)       │              0 │ dropout_2[0][0],       │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 196, 128)       │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 196, 128)       │        395,648 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 196, 128)       │              0 │ multi_head_attention_… │
│                           │                        │                │ add_1[0][0]            │
├──────────────────────

 Total params: 30,168,194 (115.08 MB)

 Trainable params: 30,168,194 (115.08 MB)

 Non-trainable params: 0 (0.00 B)

In [38]:
optimizer = keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

modelo.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    ],
)

my_callback = [keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.001, patience=5)]

# Entrenar el modelo (se asume que train_dataset y val_dataset están definidos previamente)
history = modelo.fit(train_dataset, epochs=num_epochs, batch_size=batch_size,
                     validation_data=val_dataset, validation_batch_size=batch_size,
                     callbacks=my_callback)

Epoch 1/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3762s 7s/step - accuracy: 0.5363 - loss: 12.2980 - val_accuracy: 0.6203 - val_loss: 10.5261
Epoch 2/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3300s 6s/step - accuracy: 0.5659 - loss: 10.2106 - val_accuracy: 0.6154 - val_loss: 9.0800
Epoch 3/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3280s 6s/step - accuracy: 0.6032 - loss: 8.6882 - val_accuracy: 0.6154 - val_loss: 7.4905
Epoch 4/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3323s 6s/step - accuracy: 0.6064 - loss: 7.1001 - val_accuracy: 0.6154 - val_loss: 5.9598
Epoch 5/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3344s 6s/step - accuracy: 0.6090 - loss: 5.6119 - val_accuracy: 0.6154 - val_loss: 4.5939
Epoch 6/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3289s 6s/step - accuracy: 0.6088 - loss: 4.3082 - val_accuracy: 0.6383 - val_loss: 3.4711
Epoch 7/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3333s 6s/step - accuracy: 0.6212 - loss: 3.2475 - val_accuracy: 0.6305 - val_loss: 2.5912
Epoch 8/10
540/540 ━━━━━━━━━━━━━━━━━━━━ 3227s 6s/step - accuracy: 0.6339 - loss: 2.4266

In [39]:
from tensorflow.keras.models import save_model

# Supongamos que 'model' es tu modelo ViT
modelo.save('/content/drive/My Drive/Tesis/Proyectos2/modelos/modelo_VIT_completonuevo10epocas_cambiosExp4_primerTrain.h5')


In [25]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope

with custom_object_scope({'Patches': Patches, 'PatchEncoder': PatchEncoder}):
    modelo_2 = load_model('/content/drive/My Drive/Tesis/Proyectos2/modelos/modelo_VIT_completonuevo10epocas_cambiosExp4_primerTrain.h5')

optimizer = keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=weight_decay)

modelo_2.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
    ],
)

my_callback = [keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0.001, patience=5)]


In [26]:
modelo_2.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patches_2 (Patches)       │ (None, None, 768)      │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patch_encoder_1           │ (None, 196, 128)       │        123,520 │ patches_2[0][0]        │
│ (PatchEncoder)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_34    │ (None, 196, 128)       │            256 │ patch_encoder_1[0][0]  │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_16   │ (None, 196, 128)       │        395,648 │ layer_normalization_3… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_32 (Add)              │ (None, 196, 128)       │              0 │ multi_head_attention_… │
│                           │                        │                │ patch_encoder_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_35    │ (None, 196, 128)       │            256 │ add_32[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_42 (Dense)          │ (None, 196, 256)       │         33,024 │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_56 (Dropout)      │ (None, 196, 256)       │              0 │ dense_42[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_43 (Dense)          │ (None, 196, 128)       │         32,896 │ dropout_56[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_57 (Dropout)      │ (None, 196, 128)       │              0 │ dense_43[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_33 (Add)              │ (None, 196, 128)       │              0 │ dropout_57[0][0],      │
│                           │                        │                │ add_32[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_36    │ (None, 196, 128)       │            256 │ add_33[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_17   │ (None, 196, 128)       │        395,648 │ layer_normalization_3… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_34 (Add)              │ (None, 196, 128)       │              0 │ multi_head_attention_… │
│                      

 Total params: 30,168,194 (115.08 MB)

 Trainable params: 30,168,194 (115.08 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = modelo_2.fit(train_dataset, epochs=20, batch_size=batch_size,
                     validation_data=val_dataset, validation_batch_size=batch_size,
                     callbacks=my_callback)


Epoch 1/20
489/540 ━━━━━━━━━━━━━━━━━━━━ 4:58 6s/step - accuracy: 0.6733 - loss: 0.9841

In [ ]:
from tensorflow.keras.models import save_model

# Supongamos que 'model' es tu modelo ViT
modelo_2.save('/content/drive/My Drive/Tesis/Proyectos2/modelos/modelo_VIT_completonuevo10epocas_cambiosExp4_segundoTrain.h5')


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Extraer los datos de la historia del entrenamiento
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_acc) + 1)

# Configuración para las gráficas
sns.set(style="whitegrid")
plt.figure(figsize=(14, 6))

# Gráfica de Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, label='Training Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Gráfica de Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Evaluación en el conjunto de validación
# val_metrics = modelo.evaluate(val_dataset, verbose=0)
# val_loss = val_metrics[0]
# val_acc = val_metrics[1]

# print("Validation Loss:", val_loss)
# print("Validation Accuracy:", val_acc)

# Evaluación en el conjunto de prueba
test_metrics = modelo_2.evaluate(test_dataset, verbose=0)
test_loss = test_metrics[0]
test_acc = test_metrics[1]

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Predicciones en el conjunto de validación
val_predictions = modelo_2.predict(val_dataset)
val_labels = np.concatenate([y for x, y in val_dataset], axis=0)

# Predicciones en el conjunto de prueba
test_predictions = modelo_2.predict(test_dataset)
test_labels = np.concatenate([y for x, y in test_dataset], axis=0)

# Calcular métricas
val_accuracy = accuracy_score(val_labels, np.argmax(val_predictions, axis=1))
val_precision = precision_score(val_labels, np.argmax(val_predictions, axis=1), average='binary')
val_recall = recall_score(val_labels, np.argmax(val_predictions, axis=1), average='binary')
val_f1 = f1_score(val_labels, np.argmax(val_predictions, axis=1), average='binary')
val_conf_matrix = confusion_matrix(val_labels, np.argmax(val_predictions, axis=1))

test_accuracy = accuracy_score(test_labels, np.argmax(test_predictions, axis=1))
test_precision = precision_score(test_labels, np.argmax(test_predictions, axis=1), average='binary')
test_recall = recall_score(test_labels, np.argmax(test_predictions, axis=1), average='binary')
test_f1 = f1_score(test_labels, np.argmax(test_predictions, axis=1), average='binary')
test_conf_matrix = confusion_matrix(test_labels, np.argmax(test_predictions, axis=1))

# Imprimir métricas
print('--' * 50)
print("Validacion")
print("Validation Accuracy:", val_accuracy)
print("Validation Precision:", val_precision)
print("Validation Recall:", val_recall)
print("Validation F1 Score:", val_f1)
print("Validation Confusion Matrix:\n", val_conf_matrix)

print('--' * 50)
print("Prueba")
print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1 Score:", test_f1)
print("Test Confusion Matrix:\n", test_conf_matrix)


In [ ]:
import seaborn as sns
# Función para dibujar la matriz de confusión
def plot_confusion_matrix(cm, class_names):
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(class_names)
    ax.yaxis.set_ticklabels(class_names)
    plt.show()

# Dibujar la matriz de confusión para validación
plot_confusion_matrix(val_conf_matrix, class_names=["No Glaucoma", "Glaucoma"])

# Dibujar la matriz de confusión para prueba
plot_confusion_matrix(test_conf_matrix, class_names=["No Glaucoma", "Glaucoma"])